In [0]:
spark.conf.set(
  "fs.azure.account.key.smartcitytrafficdl.dfs.core.windows.net",
  "key"
)


In [0]:
traffic = spark.read.csv(
    "abfss://processed@smartcitytrafficdl.dfs.core.windows.net/traffic/",
    header=True,
    inferSchema=True
)

weather = spark.read.csv(
    "abfss://processed@smartcitytrafficdl.dfs.core.windows.net/weather/",
    header=True,
    inferSchema=True
)

buses = spark.read.csv(
    "abfss://processed@smartcitytrafficdl.dfs.core.windows.net/buses/",
    header=True,
    inferSchema=True
)


In [0]:
from pyspark.sql.functions import col, when

df = traffic.join(weather, "Timestamp", "left")

df = df.withColumn(
    "TrafficDensity",
    col("VehicleCount") / col("AvgSpeed")
)

df = df.withColumn(
    "CongestionLevel",
    when(col("TrafficDensity") > 1.5, "High").otherwise("Low")
)

df.show()


+-------------------+--------+---------+---------+------------+--------+---------+--------------------+-----------+-------------+---------+-------------------+---------------+
|          Timestamp|SensorID| Latitude|Longitude|VehicleCount|AvgSpeed|StationID|         StationName|Temperature|Precipitation|WindSpeed|     TrafficDensity|CongestionLevel|
+-------------------+--------+---------+---------+------------+--------+---------+--------------------+-----------+-------------+---------+-------------------+---------------+
|2025-01-01 08:35:00|   S0007|28.704782|77.103178|          28|      35|    W0007|  Downtown Station 7|       35.8|          3.4|      6.1|                0.8|            Low|
|2025-01-01 09:00:00|   S0012|28.704284| 77.10107|          35|      48|    W0012|    North Station 12|       21.0|          3.6|     18.1| 0.7291666666666666|            Low|
|2025-01-01 09:20:00|   S0016|28.708931|77.103212|          32|      50|    W0016|   Uptown Station 16|       31.1|     

In [0]:
df.write.mode("overwrite").parquet(
    "abfss://curated@smartcitytrafficdl.dfs.core.windows.net/traffic_summary"
)


In [0]:
display(
  spark.read.parquet("abfss://curated@smartcitytrafficdl.dfs.core.windows.net/traffic_summary")
)


Timestamp,SensorID,Latitude,Longitude,VehicleCount,AvgSpeed,StationID,StationName,Temperature,Precipitation,WindSpeed,TrafficDensity,CongestionLevel
2025-01-02T01:20:00Z,S0208,28.708414,77.102172,48,32,W0208,Riverside Station 208,31.3,4.4,9.9,1.5,Low
2025-01-02T20:20:00Z,S0436,28.705791,77.100839,20,58,W0436,Uptown Station 436,24.1,5.1,9.4,0.3448275862068966,Low
2025-01-02T20:30:00Z,S0438,28.701967,77.097902,29,29,W0438,Riverside Station 438,29.8,3.2,16.5,1.0,Low
2025-01-04T02:20:00Z,S0796,28.705582,77.106306,20,37,W0796,Uptown Station 796,34.0,0.8,10.2,0.5405405405405406,Low
2025-01-01T10:55:00Z,S0035,28.707186,77.105024,14,52,W0035,West Station 35,23.9,8.9,19.0,0.2692307692307692,Low
2025-01-01T16:10:00Z,S0098,28.705348,77.097982,48,55,W0098,Riverside Station 98,32.0,2.2,3.7,0.8727272727272727,Low
2025-01-01T22:50:00Z,S0178,28.701899,77.106043,45,25,W0178,Riverside Station 178,20.2,3.8,3.0,1.8,High
2025-01-02T06:40:00Z,S0272,28.70726,77.10211,46,58,W0272,North Station 272,29.1,4.5,14.7,0.7931034482758621,Low
2025-01-02T09:15:00Z,S0303,28.706659,77.106709,16,45,W0303,South Station 303,27.0,1.6,13.4,0.35555555555555557,Low
2025-01-02T20:00:00Z,S0432,28.708467,77.106577,36,24,W0432,North Station 432,21.4,5.7,10.3,1.5,Low
